In [6]:
#Task 2

import numpy as np
import pandas as pd
import scipy.stats as stats
from multiprocessing import Pool


# Simulation parameters
n_simulations = 10**4  # Number of simulations to approximate the distribution
beta = 10**8  # Inverse temperature
lambda_step = 10**(-4)  # Step size for SGLD
gamma = 10**(-8)  # Regularization parameter
theta = 0  # Initial guess for theta

# Set quantile levels and number of iterations for SGLD
q = 0.95
n_iterations = 10**4
n_assets = 3  # Number of assets in the portfolio
# Define the initial weights for the portfolio
#initial_weights = np.array([1.0 / n_assets] * n_assets)

# Asset parameters for each scenario

asset_params = [
    [{'mu': 500, 'sigma': 1}, {'mu': 0, 'sigma': 10**6}, {'mu': 0, 'sigma': 10**-4}],
    [{'mu': 500, 'sigma': 1}, {'mu': 0, 'sigma': 10**6}, {'mu': 0, 'sigma': 1}],
    [{'mu': 0, 'sigma': 10**3}, {'mu': 0, 'sigma': 1}, {'mu': 0, 'sigma': 4}],
    [{'mu': 0, 'sigma': 1}, {'mu': 1, 'sigma': 4}, {'mu': 0, 'sigma': 10**-4}],
    [{'mu': 0, 'sigma': 1}, {'mu': 1, 'sigma': 4}, {'mu': 2, 'sigma': 1}]
]


In [7]:
import numpy as np

def simulate_asset_returns(mu, sigma, size):
    # Simulate asset returns with truncation to limit extreme values
    return np.clip(np.random.normal(mu, sigma, size), mu - 3*sigma, mu + 3*sigma)

# Calculate VaR and CVaR from simulated returns
def calculate_var_cvar(returns, q):
    var = np.quantile(returns, q)
    # Sort returns
    sorted_returns = np.sort(returns)
    # Find the index where VaR would be positioned in the sorted list
    var_index = np.searchsorted(sorted_returns, var, side='right')
    # CVaR is the average of the worst 1-q percent of returns
    cvar = np.mean(sorted_returns[var_index:])
    return var, cvar


def find_optimal_weights(asset_params, size, q):
    optimal_results = []
    for scenario_params in asset_params:
        mus = [param['mu'] for param in scenario_params]
        sigmas = [param['sigma'] for param in scenario_params]
        optimal_cvar = float('inf')
        optimal_weights = None
        optimal_var = None

        # Generate 100 evenly spaced weights for the first asset
        for weight_x1 in np.linspace(0, 1, 100):
            # Generate weights for the second asset based on the remaining weight
            for weight_x2 in np.linspace(0, 1 - weight_x1, 100):
                # The weight for the third asset is the remainder
                weight_x3 = 1 - weight_x1 - weight_x2

                # Simulate returns for each asset
                returns_x1 = simulate_asset_returns(mus[0], sigmas[0], size)
                returns_x2 = simulate_asset_returns(mus[1], sigmas[1], size)
                returns_x3 = simulate_asset_returns(mus[2], sigmas[2], size)

                # Calculate the portfolio returns for the given weight combination
                portfolio_returns = weight_x1 * returns_x1 + weight_x2 * returns_x2 + weight_x3 * returns_x3

                # Calculate VaR and CVaR for the portfolio
                var, cvar = calculate_var_cvar(portfolio_returns, q)

                # Check if we have a new optimal set of weights based on CVaR
                if cvar < optimal_cvar:
                    optimal_cvar = cvar
                    optimal_weights = (weight_x1, weight_x2, weight_x3)
                    optimal_var = var

        # Store the results for the scenario
        optimal_results.append({
            'optimal_weights': optimal_weights,
            'optimal_var': optimal_var,
            'optimal_cvar': optimal_cvar
        })

    return optimal_results



# Optimize the portfolio for each scenario
optimal_results = find_optimal_weights(asset_params, n_simulations, q)

# Display the results
for idx, result in enumerate(optimal_results):
    print(f"Scenario {idx+1}:")
    print(f"Optimal Weights: {result['optimal_weights']}")
    print(f"Optimal VaR: {result['optimal_var']}")
    print(f"Optimal CVaR: {result['optimal_cvar']}")



Scenario 1:
Optimal Weights: (0.0, 0.0, 1.0)
Optimal VaR: 0.0001659183526838839
Optimal CVaR: 0.00020468101973220837
Scenario 2:
Optimal Weights: (0.0, 0.0, 1.0)
Optimal VaR: 1.6597050509978113
Optimal CVaR: 2.07544320000112
Scenario 3:
Optimal Weights: (0.0, 0.9393939393939394, 0.06060606060606055)
Optimal VaR: 1.5959693112087299
Optimal CVaR: 1.9712110564237308
Scenario 4:
Optimal Weights: (0.0, 0.0, 1.0)
Optimal VaR: 0.00016502070371861646
Optimal CVaR: 0.00020855709733747787
Scenario 5:
Optimal Weights: (0.8484848484848485, 0.04591368227731863, 0.10560146923783287)
Optimal VaR: 1.6513582959994901
Optimal CVaR: 1.9780367448617395


In [9]:
# 转换结果到DataFrame，并直接拆分权重到独立的列
optimal_results_df = pd.DataFrame(optimal_results)
optimal_results_df[['Weight_X1', 'Weight_X2', 'Weight_X3']] = pd.DataFrame(
    optimal_results_df.pop('optimal_weights').tolist(), index=optimal_results_df.index
)

# 重命名其他列以匹配问题的输出
optimal_results_df.rename(columns={'optimal_var': 'VaR*', 'optimal_cvar': 'CVaR*'}, inplace=True)

# 调整列的顺序
columns_order = ['Weight_X1', 'Weight_X2', 'Weight_X3', 'VaR*', 'CVaR*']
optimal_results_df = optimal_results_df[columns_order]

# 显示DataFrame
print(optimal_results_df)



   Weight_X1  Weight_X2  Weight_X3      VaR*     CVaR*
0   0.000000   0.000000   1.000000  0.000166  0.000205
1   0.000000   0.000000   1.000000  1.659705  2.075443
2   0.000000   0.939394   0.060606  1.595969  1.971211
3   0.000000   0.000000   1.000000  0.000165  0.000209
4   0.848485   0.045914   0.105601  1.651358  1.978037
